In [2]:
# Import our dependencies
import pandas as pd
import matplotlib as plt
import numpy as np
import math
import geopy
import time
import sqlite3

In [2]:
# define column names for text file mapping
colNames = ['geonameid','name','asciiname','alternatenames',
           'latitude','longitude','feature class', 'feature code',
           'country code', 'cc2', 'admin1 code', 'admin2 code',
           'admin3 code', 'admin4 code', 'population', 'elevation',
           'dem', 'timezone', 'modification_date']

In [3]:
# load supplemental data into pandas
pop_df = pd.read_table('./Resources/population_data/TZ.txt', header=None)
pop_df.columns = colNames
pop_df.sort_values(by='population', ascending=False).head()

,geonameid,name,asciiname,alternatenames,latitude,longitude,feature class,feature code,country code,cc2,admin1 code,admin2 code,admin3 code,admin4 code,population,elevation,dem,timezone,modification_date
930,149590,United Republic of Tanzania,United Republic of Tanzania,"An Tansain,An Tansáin,Orileede Tansania,Orílẹ́...",-6.00000,35.00000,A,PCLI,TZ,NaN,0.0,NaN,NaN,NaN,56318348,NaN,829,Africa/Dar_es_Salaam,2020-03-29
11530,160260,Dar es Salaam Region,Dar es Salaam Region,"Dar es Salaam,Dar es Salaam Region,Dar-es-Salaam",-6.83523,39.19597,A,ADM1,TZ,NaN,23.0,NaN,NaN,NaN,2791063,NaN,62,Africa/Dar_es_Salaam,2011-03-17
3545,152219,Mwanza Region,Mwanza Region,"Mkoa wa Mwanza,Mwanza,Mwanza Region",-2.75000,33.08333,A,ADM1,TZ,NaN,12.0,NaN,NaN,NaN,2772509,NaN,1206,Africa/Dar_es_Salaam,2015-05-04
5683,154375,Mbeya Region,Mbeya Region,"Mbeya,Mbeya Region,Mkoa wa Mbeya",-8.20000,33.33333,A,ADM1,TZ,NaN,9.0,NaN,NaN,NaN,2707410,NaN,1315,Africa/Dar_es_Salaam,2015-05-04
11533,160263,Dar es Salaam,Dar es Salaam,"DAR,Dar Es Salaam,Dar Es Salam,Dar es Salaam,D...",-6.82349,39.26951,P,PPLA,TZ,NaN,23.0,702.0,702132.0,NaN,2698652,NaN,24,Africa/Dar_es_Salaam,2019-09-05


In [4]:
# subset data to include just city level data (admin3 code) and greater than 0 population for ranking
city_pop_df = pop_df[pop_df['admin3 code'].notnull()]
city_pop_df = city_pop_df[city_pop_df['population']>0]
city_pop_df.sort_values(by='population', ascending=False)

,geonameid,name,asciiname,alternatenames,latitude,longitude,feature class,feature code,country code,cc2,admin1 code,admin2 code,admin3 code,admin4 code,population,elevation,dem,timezone,modification_date
11533,160263,Dar es Salaam,Dar es Salaam,"DAR,Dar Es Salaam,Dar Es Salam,Dar es Salaam,D...",-6.82349,39.26951,P,PPLA,TZ,NaN,23.0,702.0,702132.0,NaN,2698652,NaN,24,Africa/Dar_es_Salaam,2019-09-05
536,149193,Unguja Ukuu Kaebona,Unguja Ukuu Kaebona,"Unguja Ukuu,Unguja Ukuu Kaebona",-6.27452,39.37486,A,ADM3,TZ,NaN,21.0,5201.0,5201381.0,NaN,1060416,NaN,32,Africa/Dar_es_Salaam,2016-07-25
73,148726,Zanzibar Island,Zanzibar Island,"Ile de Zanzibar,Menuthias,Unguja,Unguja Island...",-6.15557,39.34170,T,ISL,TZ,NaN,21.0,5201.0,5201181.0,NaN,896721,NaN,39,Africa/Dar_es_Salaam,2019-04-13
3549,152224,Mwanza,Mwanza,"MWZ,Muansa,Muanza,Muvanzo,Mvanza,Mwamza,Mwansa...",-2.51667,32.90000,P,PPLA,TZ,NaN,12.0,1903.0,1903042.0,NaN,436801,NaN,1144,Africa/Dar_es_Salaam,2019-09-05
77,148730,Zanzibar,Zanzibar,"Ilu Zanzibar,Poli tis Zanzivaris,Sansibar,Sans...",-6.16394,39.19793,P,PPLA,TZ,NaN,25.0,5302.0,5302272.0,NaN,403658,NaN,14,Africa/Dar_es_Salaam,2019-09-05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7880,156588,Koani Ndogo,Koani Ndogo,Koani Ndogo,-6.13333,39.28333,P,PPL,TZ,NaN,21.0,5201.0,5201093.0,NaN,2311,NaN,44,Africa/Dar_es_Salaam,2018-12-04
7881,156589,Koani,Koani,Koani,-6.13333,39.28333,P,PPLA,TZ,NaN,21.0,5201.0,5201093.0,NaN,2211,NaN,44,Africa/Dar_es_Salaam,2016-07-26
3050,151720,Nganane,Nganane,NaN,-6.40000,39.55000,P,PPL,TZ,NaN,21.0,5202.0,5202041.0,NaN,1929,NaN,31,Africa/Dar_es_Salaam,2016-07-26
14350,6615413,Hadzabe encampment,Hadzabe encampment,NaN,-3.52810,35.43537,S,HUTS,TZ,NaN,26.0,204.0,204061.0,NaN,20,NaN,1135,Africa/Dar_es_Salaam,2016-07-26


In [5]:
# Test the creation of the geocode object
from geopy.geocoders import get_geocoder_for_service
get_geocoder_for_service("nominatim")

geopy.geocoders.osm.Nominatim

In [6]:
# Display lat/long of the City dataset
city_pop_df[['latitude','longitude']]

,latitude,longitude
73,-6.15557,39.34170
77,-6.16394,39.19793
187,-5.05589,39.72938
286,-9.10806,32.93472
330,-7.00667,39.29849
...,...,...
14350,-3.52810,35.43537
14362,-5.72651,39.29870
15048,-3.37146,36.79356
15051,-6.17526,35.79266


In [7]:
# Define function to calculate distance between two GPS points - return distance in km

import math

def haversine(coord1, coord2):
    R = 6372800  # Earth radius in meters
    lat1, lon1 = coord1
    lat2, lon2 = coord2
    
    phi1, phi2 = math.radians(lat1), math.radians(lat2) 
    dphi       = math.radians(lat2 - lat1)
    dlambda    = math.radians(lon2 - lon1)
    
    a = math.sin(dphi/2)**2 + \
        math.cos(phi1)*math.cos(phi2)*math.sin(dlambda/2)**2
    
    return 2*R*math.atan2(math.sqrt(a), math.sqrt(1 - a))/1000

In [8]:
haversine([51.5073219,-0.1276474],[52.5170365,13.3888599])

930.7232019867425

In [9]:
# create a combined pandas dataframe with lat/lon

ref_coord = [51.5073219,-0.1276474]

# city_pop_df['geo_loc'] = city_pop_df['latitude'].astype(str) + "," + city_pop_df['longitude'].astype(str)
city_pop_df['geo_loc'] = city_pop_df[['latitude','longitude']].values.tolist()
city_pop_df['distance'] = city_pop_df['geo_loc'].apply(haversine,args=(ref_coord,))

city_pop_df['distance']

73       7431.003832
77       7425.010265
187      7339.388491
286      7449.213099
330      7514.464783
            ...     
14350    6984.319635
14362    7385.888755
15048    7030.724781
15051    7269.624895
19346    7498.313976
Name: distance, Length: 280, dtype: float64

In [10]:
import sqlite3

# Read sqlite query results into a pandas DataFrame
con = sqlite3.connect("Tanzania_Water_Pump.db")
water_df = pd.read_sql_query("SELECT * from pump_it_up_training_set", con)

con.close()

# Create a combined column with lat/lon and drop long where 0
water_df['longitude'] = water_df['longitude'].astype(float)
water_df['geo_loc'] = water_df[['latitude','longitude']].values.tolist()

water_test = water_df.head(10)

# Verify that result of SQL query is stored in the dataframe
water_test

,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,...,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group,status_group,geo_loc
0,0,0.0,2012-11-13,Tasaf,0,TASAF,33.125828,-5.118154,Mratibu,0,...,milky,enough,enough,shallow well,shallow well,groundwater,hand pump,hand pump,non functional,"[-5.11815407, 33.1258283]"
1,1,0.0,2011-03-05,Shipo,1978,SHIPO,34.770717,-9.395642,none,0,...,good,enough,enough,shallow well,shallow well,groundwater,hand pump,hand pump,functional,"[-9.39564152, 34.77071669]"
2,2,0.0,2011-03-27,Lvia,0,LVIA,36.115056,-6.279268,Bombani,0,...,good,insufficient,insufficient,machine dbh,borehole,groundwater,communal standpipe multiple,communal standpipe,functional,"[-6.27926803, 36.11505595]"
3,3,10.0,2013-06-03,Germany Republi,1639,CES,37.147432,-3.187555,Area 7 Namba 5,0,...,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe,functional,"[-3.18755455, 37.14743219]"
4,4,0.0,2011-03-22,Cmsr,0,CMSR,36.164893,-6.099289,Ezeleda,0,...,good,dry,dry,shallow well,shallow well,groundwater,hand pump,hand pump,non functional,"[-6.09928949, 36.16489341]"
5,5,50.0,2011-02-26,Private,28,Private,39.286124,-6.972403,Kwa Namaj,0,...,good,enough,enough,machine dbh,borehole,groundwater,communal standpipe multiple,communal standpipe,functional,"[-6.97240273, 39.28612441]"
6,6,0.0,2012-10-20,Government Of Tanzania,0,Government,33.229880,-3.852983,Mission,0,...,good,seasonal,seasonal,shallow well,shallow well,groundwater,other,other,non functional,"[-3.85298271, 33.229879700000005]"
7,7,0.0,2011-03-23,Water,0,Gover,36.313619,-6.719257,Itawi,0,...,milky,insufficient,insufficient,spring,spring,groundwater,improved spring,improved spring,functional,"[-6.71925716, 36.31361889]"
8,8,0.0,2011-04-07,Water,0,Commu,35.939445,-6.014358,Kwa Chiswagala,0,...,good,dry,dry,machine dbh,borehole,groundwater,communal standpipe multiple,communal standpipe,non functional,"[-6.0143582, 35.93944483]"
9,9,0.0,2011-07-12,Ded,0,DED,31.693371,-2.530703,Shule Ya Msingi Nyamirembe A,0,...,good,insufficient,insufficient,machine dbh,borehole,groundwater,communal standpipe multiple,communal standpipe,non functional,"[-2.53070326, 31.69337116]"


In [11]:
# Custom apply function for finding 3 closest city distance and population
def compare_distance(ref_df, city_df, pos):

    # use GPS coordinates of current water table and all Cities to calc distance
    city_df['distance'] = city_df['geo_loc'].apply(haversine,args=(ref_df,))
    
    # Sort a subset dataframe by distance ascending
    sorted_cities = city_df[['distance','population']].sort_values(by=['distance']).iloc[:3]
    
    # Slice custom values for the desired position
    city = sorted_cities.iloc[[pos]]
    colDistance = 'distance' + str(pos)
    colPop = 'population' + str(pos)
    
    city.columns = [colDistance,colPop]
    
    # Return as series so it can add a new column
    return pd.Series([city[colDistance].values[0],city[colPop].values[0]])

In [29]:
# Custom apply function for finding 3 closest city distance and population
def compare_distance_all(ref, city_df):
    
    # use GPS coordinates of current water table and all Cities to calc distance
    city_df['distance'] = city_df['geo_loc'].apply(haversine,args=(ref,))
    
    # Sort a subset dataframe by distance ascending
    sorted_cities = city_df[['distance','population']].sort_values(by=['distance']).iloc[:3]
    
    # Slice custom values for the desired position
    city1 = sorted_cities.iloc[[0]]
    city1.columns = ['distance1','population1']
    
    city2 = sorted_cities.iloc[[1]]
    city2.columns = ['distance2','population2']
    
    city3 = sorted_cities.iloc[[2]]
    city3.columns = ['distance3','population3']
    
    city_series = [city1['distance1'].values[0],city1['population1'].values[0],
                  city2['distance2'].values[0],city2['population2'].values[0],
                  city3['distance3'].values[0],city3['population3'].values[0]]

    # Return as series so it can add a new column
    return pd.Series(city_series)

In [30]:
# Add the three closest cities to the test dataframe
water_test[['distance1','population1',
            'distance2','population2',
            'distance3','population3']] = water_test['geo_loc'].apply(compare_distance_all,
                                                            args=(city_pop_df,))

water_test

,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,...,waterpoint_type,waterpoint_type_group,status_group,geo_loc,distance1,population1,distance2,population2,distance3,population3
0,0,0.0,2012-11-13,Tasaf,0,TASAF,33.125828,-5.118154,Mratibu,0,...,hand pump,hand pump,non functional,"[-5.11815407, 33.1258283]",35.033967,145292.0,43.859263,32900.0,65.659579,15320.0
1,1,0.0,2011-03-05,Shipo,1978,SHIPO,34.770717,-9.395642,none,0,...,hand pump,hand pump,functional,"[-9.39564152, 34.77071669]",5.169916,46724.0,42.274055,19040.0,51.339210,15168.0
2,2,0.0,2011-03-27,Lvia,0,LVIA,36.115056,-6.279268,Bombani,0,...,communal standpipe multiple,communal standpipe,functional,"[-6.27926803, 36.11505595]",27.054749,5527.0,34.494481,11840.0,37.477470,10000.0
3,3,10.0,2013-06-03,Germany Republi,1639,CES,37.147432,-3.187555,Area 7 Namba 5,0,...,communal standpipe,communal standpipe,functional,"[-3.18755455, 37.14743219]",21.108603,22839.0,27.433508,156959.0,38.571507,18726.0
4,4,0.0,2011-03-22,Cmsr,0,CMSR,36.164893,-6.099289,Ezeleda,0,...,hand pump,hand pump,non functional,"[-6.09928949, 36.16489341]",15.549644,5527.0,30.011861,11840.0,42.023175,10000.0
5,5,50.0,2011-02-26,Private,28,Private,39.286124,-6.972403,Kwa Namaj,0,...,communal standpipe multiple,communal standpipe,functional,"[-6.97240273, 39.28612441]",4.048531,23307.0,14.920088,8.0,16.664311,2698652.0
6,6,0.0,2012-10-20,Government Of Tanzania,0,Government,33.229880,-3.852983,Mission,0,...,other,other,non functional,"[-3.85298271, 33.229879700000005]",4.731621,22082.0,29.882329,107362.0,33.321601,21094.0
7,7,0.0,2011-03-23,Water,0,Gover,36.313619,-6.719257,Itawi,0,...,improved spring,improved spring,functional,"[-6.71925716, 36.31361889]",5.867228,19526.0,45.150167,19806.0,58.867328,11840.0
8,8,0.0,2011-04-07,Water,0,Commu,35.939445,-6.014358,Kwa Chiswagala,0,...,communal standpipe multiple,communal standpipe,non functional,"[-6.0143582, 35.93944483]",11.115637,5527.0,24.162570,10000.0,28.238465,180541.0
9,9,0.0,2011-07-12,Ded,0,DED,31.693371,-2.530703,Shule Ya Msingi Nyamirembe A,0,...,communal standpipe multiple,communal standpipe,non functional,"[-2.53070326, 31.69337116]",14.445214,27776.0,44.180072,34038.0,58.408875,52870.0


In [31]:
# Add the three closest cities to the dataframe
water_df[['distance1','population1',
            'distance2','population2',
            'distance3','population3']] = water_df['geo_loc'].apply(compare_distance_all,
                                                            args=(city_pop_df,))

water_df.sample(5)

,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,...,waterpoint_type,waterpoint_type_group,status_group,geo_loc,distance1,population1,distance2,population2,distance3,population3
8434,10532,0.0,2013-02-04,Dwsp,0,DWE,0.000000,-2.000000e-08,Mkombozi,0,...,hand pump,hand pump,functional needs repair,"[-2e-08, 0.0]",3335.612147,164268.0,3338.285380,16822.0,3383.076729,37349.0
26923,33590,2000.0,2011-03-06,Danida,1405,DANID,36.092589,-7.563910e+00,none,0,...,communal standpipe,communal standpipe,functional,"[-7.56390959, 36.09258907]",13.972517,29193.0,48.799803,111820.0,56.645359,18010.0
3978,4994,0.0,2011-02-21,Roman,1295,Roman,34.630298,-9.972548e+00,none,0,...,communal standpipe,communal standpipe,non functional,"[-9.97254811, 34.63029767]",24.453298,15168.0,71.047723,46724.0,77.142285,15009.0
32720,40925,20.0,2011-08-03,Government Of Tanzania,387,RWE,37.435324,-6.298465e+00,Kanisani,0,...,communal standpipe,communal standpipe,functional,"[-6.29846549, 37.4353241]",11.072782,18753.0,36.863974,17861.0,63.255024,250902.0
50617,63345,500.0,2011-01-04,Unicef,1572,DWE,34.547080,-8.994904e+00,Kwa Semojo,0,...,communal standpipe,communal standpipe,functional,"[-8.99490445, 34.54708043]",11.282267,18870.0,27.889799,19040.0,35.342161,53003.0


In [32]:
# Export to CSV with merged supplemental data
water_df.to_csv("water_pump_closest_cities.csv")

In [5]:
# Import the supplemneted CSV into the SQLlite database

# load data
df = pd.read_csv('water_pump_closest_cities.csv')

# strip whitespace from headers
df.columns = df.columns.str.strip()

con = sqlite3.connect("Tanzania_Water_Pump.db")

# drop data into database
df.to_sql("Water_Pump_Status_Complete", con)

con.close()

C:\Users\mattg\Anaconda3\envs\PythonData\lib\site-packages\pandas\core\generic.py:2662: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  method=method,
